# EfficientNet-B0 Experimentation on Cityscapes Dataset

This notebook implements a series of experiments to evaluate and improve the performance of EfficientNet-B0 on the Cityscapes dataset.

## Overview

1. **Baseline Experiment**: Train EfficientNet-B0 with standard settings
2. **Modified Models**:
   - Add CBAM (Convolutional Block Attention Module)
   - Switch to Mish activation function
   - Add DeeplabV3+ segmentation head
3. **Comparative Analysis**: Compare and analyze the results across all models

## 1. Environment Setup

First, let's import all necessary libraries for our experiments.

In [ ]:
# Import standard libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
from tqdm.notebook import tqdm

# PyTorch imports
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, random_split
import torchvision
from torchvision import transforms, models
from efficientnet_pytorch import EfficientNet

# Set seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Check if CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

## 2. Data Preparation

### 2.1 Loading Cityscapes Dataset

We'll load the Cityscapes dataset and prepare it for our experiments. First, we need to clone the Cityscapes repository to access the scripts.

In [ ]:
# Clone the Cityscapes repository if not already present
%git clone https://github.com/mcordts/cityscapesScripts.git
%pip install -e cityscapesScripts

In [ ]:
# Import Cityscapes helper functions
from cityscapesscripts.helpers.labels import trainId2label, id2label

# Define data transforms
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),  # EfficientNet-B0 input size
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Define path to the Cityscapes dataset
# Update this path to where your Cityscapes data is located
cityscapes_root = 'path/to/cityscapes'

# Create dataset using built-in Cityscapes dataset class from torchvision
from torchvision.datasets import Cityscapes

# Load the dataset
train_dataset = Cityscapes(
    root=cityscapes_root,
    split='train',
    mode='fine',
    target_type='semantic',
    transform=train_transform,
    target_transform=None
)

val_dataset = Cityscapes(
    root=cityscapes_root,
    split='val',
    mode='fine',
    target_type='semantic',
    transform=val_test_transform,
    target_transform=None
)

In [ ]:
# Display dataset information
print(f"Train dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")

# Split validation into validation and test sets
val_size = int(len(val_dataset) * 0.5)
test_size = len(val_dataset) - val_size
val_dataset, test_dataset = random_split(val_dataset, [val_size, test_size])

print(f"After splitting - Validation dataset size: {len(val_dataset)}")
print(f"After splitting - Test dataset size: {len(test_dataset)}")

# Create data loaders
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

# Show a sample image from the dataset
def show_sample(dataset, idx=0):
    img, label = dataset[idx]
    
    # Denormalize the image
    mean = torch.tensor([0.485, 0.456, 0.406])
    std = torch.tensor([0.229, 0.224, 0.225])
    img_denorm = img * std[:, None, None] + mean[:, None, None]
    
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.imshow(img_denorm.permute(1, 2, 0).numpy())
    plt.title('Image')
    
    plt.subplot(1, 2, 2)
    plt.imshow(label)
    plt.title('Segmentation Mask')
    plt.show()

show_sample(train_dataset, idx=np.random.randint(len(train_dataset)))

## 3. Baseline Model: EfficientNet-B0

We'll now set up our baseline model using EfficientNet-B0.

In [ ]:
class EfficientNetB0Classifier(nn.Module):
    def __init__(self, num_classes=19):  # Cityscapes has 19 classes with trainId
        super(EfficientNetB0Classifier, self).__init__()
        # Load the pre-trained EfficientNet-B0 model
        self.efficient_net = EfficientNet.from_pretrained('efficientnet-b0')
        
        # Replace the classifier with a new one for the number of classes in Cityscapes
        in_features = self.efficient_net._fc.in_features
        self.efficient_net._fc = nn.Linear(in_features, num_classes)
    
    def forward(self, x):
        return self.efficient_net(x)

# Initialize the baseline model
baseline_model = EfficientNetB0Classifier().to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(baseline_model.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3, factor=0.1)

In [ ]:
def train_one_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    running_corrects = 0
    processed_data = 0
    
    for inputs, labels in tqdm(dataloader, desc="Training"):
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimize
        loss.backward()
        optimizer.step()
        
        # Statistics
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
        processed_data += inputs.size(0)
        
    train_loss = running_loss / processed_data
    train_acc = running_corrects.double() / processed_data
    
    return train_loss, train_acc.item()

def evaluate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    running_corrects = 0
    processed_size = 0
    
    with torch.no_grad():
        for inputs, labels in tqdm(dataloader, desc="Evaluating"):
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            # Forward pass
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)
            
            # Statistics
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
            processed_size += inputs.size(0)
    
    eval_loss = running_loss / processed_size
    eval_acc = running_corrects.double() / processed_size
    
    return eval_loss, eval_acc.item()

In [ ]:
# Training loop for baseline model
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=10):
    history = {
        'train_loss': [],
        'train_acc': [],
        'val_loss': [],
        'val_acc': []
    }
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)
        
        # Train phase
        train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, device)
        print(f'Train Loss: {train_loss:.4f} Acc: {train_acc:.4f}')
        
        # Validation phase
        val_loss, val_acc = evaluate(model, val_loader, criterion, device)
        print(f'Val Loss: {val_loss:.4f} Acc: {val_acc:.4f}')
        
        # Update learning rate
        scheduler.step(val_loss)
        
        # Deep copy the model if it's the best
        if val_acc > best_acc:
            best_acc = val_acc
            best_model_wts = copy.deepcopy(model.state_dict())
        
        # Update history
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
        
        print()
    
    # Load best model weights
    model.load_state_dict(best_model_wts)
    return model, history

import copy

# Train the baseline model
baseline_model_trained, baseline_history = train_model(
    baseline_model, 
    train_loader, 
    val_loader, 
    criterion, 
    optimizer, 
    scheduler,
    num_epochs=10
)

In [ ]:
# Evaluate the baseline model on test set
test_loss, test_acc = evaluate(baseline_model_trained, test_loader, criterion, device)
print(f'Baseline Model - Test Loss: {test_loss:.4f} Acc: {test_acc:.4f}')

In [ ]:
# Visualize the training history
def plot_training_history(history, title):
    epochs = range(1, len(history['train_loss'])+1)
    
    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    plt.plot(epochs, history['train_loss'], 'bo-', label='Training Loss')
    plt.plot(epochs, history['val_loss'], 'ro-', label='Validation Loss')
    plt.title(f'{title} - Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(epochs, history['train_acc'], 'bo-', label='Training Accuracy')
    plt.plot(epochs, history['val_acc'], 'ro-', label='Validation Accuracy')
    plt.title(f'{title} - Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    
    plt.tight_layout()
    plt.show()

# Visualize baseline model training history
plot_training_history(baseline_history, 'Baseline EfficientNet-B0')

## 4. Modified Models

### 4.1 EfficientNet-B0 with CBAM (Convolutional Block Attention Module)

CBAM enhances the representational power by focusing on important features and suppressing unnecessary ones.

In [ ]:
# Implementing CBAM 
class ChannelAttention(nn.Module):
    def __init__(self, in_planes, ratio=16):
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
           
        self.fc = nn.Sequential(
            nn.Conv2d(in_planes, in_planes // ratio, 1, bias=False),
            nn.ReLU(),
            nn.Conv2d(in_planes // ratio, in_planes, 1, bias=False)
        )
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = self.fc(self.avg_pool(x))
        max_out = self.fc(self.max_pool(x))
        out = avg_out + max_out
        return self.sigmoid(out)

class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()
        padding = kernel_size // 2
        self.conv = nn.Conv2d(2, 1, kernel_size, padding=padding, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        concat = torch.cat([avg_out, max_out], dim=1)
        out = self.conv(concat)
        return self.sigmoid(out)

class CBAM(nn.Module):
    def __init__(self, in_planes, ratio=16, kernel_size=7):
        super(CBAM, self).__init__()
        self.channel_att = ChannelAttention(in_planes, ratio)
        self.spatial_att = SpatialAttention(kernel_size)

    def forward(self, x):
        x = x * self.channel_att(x)
        x = x * self.spatial_att(x)
        return x

# EfficientNet-B0 with CBAM attention
class EfficientNetB0WithCBAM(nn.Module):
    def __init__(self, num_classes=19):
        super(EfficientNetB0WithCBAM, self).__init__()
        self.efficient_net = EfficientNet.from_pretrained('efficientnet-b0')
        in_features = self.efficient_net._fc.in_features
        
        # Add CBAM at the end of feature extraction
        self.cbam = CBAM(in_features)
        
        # Replace classifier
        self.efficient_net._fc = nn.Linear(in_features, num_classes)
    
    def forward(self, x):
        # Extract features before the final FC layer
        features = self.efficient_net.extract_features(x)
        
        # Apply CBAM
        features_with_attention = self.cbam(features)
        
        # Continue with the rest of EfficientNet forward pass
        x = self.efficient_net._avg_pooling(features_with_attention)
        x = x.flatten(start_dim=1)
        x = self.efficient_net._dropout(x)
        x = self.efficient_net._fc(x)
        
        return x

# Initialize the CBAM model
cbam_model = EfficientNetB0WithCBAM().to(device)

# Define optimizer for CBAM model
cbam_optimizer = optim.SGD(cbam_model.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-4)
cbam_scheduler = optim.lr_scheduler.ReduceLROnPlateau(cbam_optimizer, 'min', patience=3, factor=0.1)

In [ ]:
# Train the CBAM model
cbam_model_trained, cbam_history = train_model(
    cbam_model, 
    train_loader, 
    val_loader, 
    criterion, 
    cbam_optimizer, 
    cbam_scheduler,
    num_epochs=10
)

# Evaluate CBAM model on test set
cbam_test_loss, cbam_test_acc = evaluate(cbam_model_trained, test_loader, criterion, device)
print(f'CBAM Model - Test Loss: {cbam_test_loss:.4f} Acc: {cbam_test_acc:.4f}')

# Visualize CBAM model training history
plot_training_history(cbam_history, 'EfficientNet-B0 with CBAM')

### 4.2 EfficientNet-B0 with Mish Activation Function

Mish is a self-regularized non-monotonic activation function that often outperforms ReLU and its variants.

In [ ]:
# Implementing Mish activation
class Mish(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return x * torch.tanh(F.softplus(x))

# EfficientNet-B0 with Mish activation
class EfficientNetB0WithMish(nn.Module):
    def __init__(self, num_classes=19):
        super(EfficientNetB0WithMish, self).__init__()
        self.efficient_net = EfficientNet.from_pretrained('efficientnet-b0')
        
        # Replace all activation functions with Mish
        self._replace_relu_with_mish(self.efficient_net)
        
        # Replace classifier
        in_features = self.efficient_net._fc.in_features
        self.efficient_net._fc = nn.Linear(in_features, num_classes)

    def _replace_relu_with_mish(self, model):
        for name, module in model.named_children():
            if isinstance(module, nn.ReLU):
                setattr(model, name, Mish())
            else:
                self._replace_relu_with_mish(module)
    
    def forward(self, x):
        return self.efficient_net(x)

# Initialize the Mish model
mish_model = EfficientNetB0WithMish().to(device)

# Define optimizer for Mish model
mish_optimizer = optim.SGD(mish_model.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-4)
mish_scheduler = optim.lr_scheduler.ReduceLROnPlateau(mish_optimizer, 'min', patience=3, factor=0.1)

In [ ]:
# Train the Mish model
mish_model_trained, mish_history = train_model(
    mish_model, 
    train_loader, 
    val_loader, 
    criterion, 
    mish_optimizer, 
    mish_scheduler,
    num_epochs=10
)

# Evaluate Mish model on test set
mish_test_loss, mish_test_acc = evaluate(mish_model_trained, test_loader, criterion, device)
print(f'Mish Model - Test Loss: {mish_test_loss:.4f} Acc: {mish_test_acc:.4f}')

# Visualize Mish model training history
plot_training_history(mish_history, 'EfficientNet-B0 with Mish')

### 4.3 EfficientNet-B0 with DeeplabV3+ Segmentation Head

DeepLabV3+ is a semantic segmentation architecture that combines atrous convolution with encoder-decoder structure.

In [ ]:
# Implementing DeeplabV3+ segmentation head
class ASPP(nn.Module):
    def __init__(self, in_channels, out_channels, rates=[6, 12, 18]):
        super(ASPP, self).__init__()
        
        self.aspp = nn.ModuleList()
        
        # 1x1 convolution
        self.aspp.append(nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU()
        ))
        
        # Atrous convolutions
        for rate in rates:
            self.aspp.append(nn.Sequential(
                nn.Conv2d(in_channels, out_channels, 3, padding=rate, dilation=rate, bias=False),
                nn.BatchNorm2d(out_channels),
                nn.ReLU()
            ))
        
        # Global average pooling
        self.global_avg_pool = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(in_channels, out_channels, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU()
        )
        
        # Output layer
        self.output = nn.Sequential(
            nn.Conv2d(out_channels * (len(rates) + 2), out_channels, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU()
        )
        
    def forward(self, x):
        size = x.size()[2:]
        
        outputs = []
        for module in self.aspp:
            outputs.append(module(x))
        
        # Process global average pooling branch
        gap_output = self.global_avg_pool(x)
        gap_output = F.interpolate(gap_output, size=size, mode='bilinear', align_corners=True)
        outputs.append(gap_output)
        
        # Concatenate and process through output layer
        x = torch.cat(outputs, dim=1)
        return self.output(x)

class DeepLabV3Plus(nn.Module):
    def __init__(self, base_model, num_classes=19, output_stride=16):
        super(DeepLabV3Plus, self).__init__()
        self.backbone = base_model
        in_features = self.backbone._fc.in_features
        
        # ASPP module
        self.aspp = ASPP(in_features, 256)
        
        # Decoder
        self.decoder = nn.Sequential(
            nn.Conv2d(256, 256, 3, padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256, num_classes, 1)
        )
    
    def forward(self, x):
        input_size = x.size()[2:]
        
        # Extract features
        features = self.backbone.extract_features(x)
        
        # Apply ASPP
        x = self.aspp(features)
        
        # Decoder
        x = self.decoder(x)
        
        # Upsampling to original size
        x = F.interpolate(x, size=input_size, mode='bilinear', align_corners=True)
        
        return x

# Initialize the DeepLabV3+ model
base_model = EfficientNet.from_pretrained('efficientnet-b0')
deeplabv3_model = DeepLabV3Plus(base_model).to(device)

# Define optimizer for DeepLabV3+ model
deeplabv3_optimizer = optim.SGD(deeplabv3_model.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-4)
deeplabv3_scheduler = optim.lr_scheduler.ReduceLROnPlateau(deeplabv3_optimizer, 'min', patience=3, factor=0.1)

In [ ]:
# Train the DeepLabV3+ model
deeplabv3_model_trained, deeplabv3_history = train_model(
    deeplabv3_model, 
    train_loader, 
    val_loader, 
    criterion, 
    deeplabv3_optimizer, 
    deeplabv3_scheduler,
    num_epochs=10
)

# Evaluate DeepLabV3+ model on test set
deeplabv3_test_loss, deeplabv3_test_acc = evaluate(deeplabv3_model_trained, test_loader, criterion, device)
print(f'DeepLabV3+ Model - Test Loss: {deeplabv3_test_loss:.4f} Acc: {deeplabv3_test_acc:.4f}')

# Visualize DeepLabV3+ model training history
plot_training_history(deeplabv3_history, 'EfficientNet-B0 with DeepLabV3+')

## 5. Results Comparison and Analysis

Let's compare the performance of all model variants across various metrics.

In [ ]:
# Create comparison table
results = {
    'Model': ['Baseline', 'With CBAM', 'With Mish', 'With DeepLabV3+'],
    'Test Accuracy': [test_acc, cbam_test_acc, mish_test_acc, deeplabv3_test_acc],
    'Test Loss': [test_loss, cbam_test_loss, mish_test_loss, deeplabv3_test_loss]
}

results_df = pd.DataFrame(results)
print("Model Performance Comparison:")
display(results_df)

# Visualize comparison
plt.figure(figsize=(10, 6))

x = np.arange(len(results['Model']))
width = 0.35

plt.bar(x - width/2, results['Test Accuracy'], width, label='Test Accuracy')
plt.bar(x + width/2, results['Test Loss'], width, label='Test Loss')

plt.xlabel('Model')
plt.title('Performance Comparison of EfficientNet-B0 Variants')
plt.xticks(x, results['Model'], rotation=45)
plt.legend()

plt.tight_layout()
plt.show()